# Data Preparation

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unzip dataset 
!unzip "/content/drive/MyDrive/Miniproject - Sem 6/HandGestures.zip" -d "/content/HandGestures/" &> /dev/null 

Streaming output truncated to the last 5000 lines.
  inflating: /content/HandGestures/train/train/4/46.jpg  
  inflating: /content/HandGestures/train/train/4/460.jpg  
  inflating: /content/HandGestures/train/train/4/461.jpg  
  inflating: /content/HandGestures/train/train/4/462.jpg  
  inflating: /content/HandGestures/train/train/4/463.jpg  
  inflating: /content/HandGestures/train/train/4/464.jpg  
  inflating: /content/HandGestures/train/train/4/465.jpg  
  inflating: /content/HandGestures/train/train/4/466.jpg  
  inflating: /content/HandGestures/train/train/4/467.jpg  
  inflating: /content/HandGestures/train/train/4/468.jpg  
  inflating: /content/HandGestures/train/train/4/469.jpg  
  inflating: /content/HandGestures/train/train/4/47.jpg  
  inflating: /content/HandGestures/train/train/4/470.jpg  
  inflating: /content/HandGestures/train/train/4/471.jpg  
  inflating: /content/HandGestures/train/train/4/472.jpg  
  inflating: /content/HandGestures/train/train/4/473.jpg  
  infla

In [ ]:
import os, shutil, numpy as np

# Define directories and validation data size
parent_path = '/content/HandGestures/'
train_path = '/content/HandGestures/train/train/'
validation_path = '/content/HandGestures/validation/'

os.makedirs(os.path.join(parent_path, 'validation'))

# Set 20% data for validation
val_size = 0.2

In [ ]:
# Remove extra directories from training directory
for f in os.scandir(train_path):
    if len(f.path) == 36:
        print(f'Removing directory: "{f.name}"')
        shutil.rmtree(f.path)

Removing directory: "12"
Removing directory: "18"
Removing directory: "19"
Removing directory: "10"
Removing directory: "13"
Removing directory: "17"
Removing directory: "16"
Removing directory: "15"
Removing directory: "11"
Removing directory: "14"


In [ ]:
# Create directory for validation data (20%)
for folder in os.listdir(train_path):
    cwd = os.path.join(train_path, folder)
    imageList = os.listdir(cwd)
    print(f'Total images found in "{folder}": {len(imageList)}')
    valImages = np.random.choice(imageList,size=int(len(imageList) * val_size), replace=False)
    os.mkdir(os.path.join(parent_path, os.path.join('validation', folder)))
    for image in valImages:
        os.rename(parent_path+'/train/train/'+folder+'/'+image, parent_path+'/validation/'+folder+'/'+image)

Total images found in "2": 900
Total images found in "7": 900
Total images found in "1": 900
Total images found in "6": 900
Total images found in "4": 900
Total images found in "8": 900
Total images found in "0": 900
Total images found in "3": 900
Total images found in "5": 900
Total images found in "9": 900


In [ ]:
print('Images in each folder in training directory: ', len(os.listdir(train_path+'/0')))
print('Images in each folder in validation directory: ', len(os.listdir(parent_path+'/validation/0')))

Images in each folder in training directory:  720
Images in each folder in validation directory:  180


# Defining CNN Architecture

In [ ]:
from keras.applications.mobilenet import MobileNet

# Input size of images (50, 50)
img_rows, img_cols = 50, 50 

# Re-load the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

17235968/17225924 [==============================] - 0s 0us/step
0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization F

In [ ]:
# Top layer to put on MobileNet
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model

# Set no. of classes to 10
num_classes = 10

FC_Head = lw(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 25, 25, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 25, 25, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 25, 25, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 25, 25, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 25, 25, 32)       128       
 ation)                                                      

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Data Augmentation 
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Set batch size
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 7200 images belonging to 10 classes.
Found 1800 images belonging to 10 classes.


# Model Training

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
        
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Miniproject - Sem 6/gesture_classifier.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# Call backs in a callback list
callbacks = [earlystop, checkpoint]

model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(learning_rate = 0.001),
              metrics = ['accuracy'])

# Number of training and validation samples here
nb_train_samples = 7200
nb_validation_samples = 1800
 
epochs = 10
batch_size = 32

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/10
222/225 [============================>.] - ETA: 0s - loss: 0.4190 - accuracy: 0.8516
Epoch 1: val_loss improved from inf to 0.30069, saving model to /content/drive/MyDrive/Miniproject - Sem 6/gesture_classifier.h5
225/225 [==============================] - 18s 24ms/step - loss: 0.4170 - accuracy: 0.8525 - val_loss: 0.3007 - val_accuracy: 0.9007
Epoch 2/10
223/225 [============================>.] - ETA: 0s - loss: 0.2092 - accuracy: 0.9242
Epoch 2: val_loss improved from 0.30069 to 0.19011, saving model to /content/drive/MyDrive/Miniproject - Sem 6/gesture_classifier.h5
225/225 [==============================] - 5s 21ms/step - loss: 0.2088 - accuracy: 0.9242 - val_loss: 0.1901 - val_accuracy: 0.9392
Epoch 3/10
224/225 [============================>.] - ETA: 0s - loss: 0.1769 - accuracy: 0.9375
Epoch 3: val_loss did not improve from 0.19011
225/225 [==============================] - 4s 19ms/step - loss: 0.1765 - accuracy: 0.9375 - val_loss: 0.2186 - val_accuracy: 0.9319
Epoch 